# STEAM DATA TREND (2024)


# DESCRIPTION

We want to chart:
Average playtime vs publisher class
Average playtime vs review score
Average playtime vs price

# MISC

-- Data set did not specify currency, had to double check to make sure  
-- Some games might have been translated indepdently, steamId still valid  
-- Some games are incorrectly labeled or missing information (WWE 2k24 was deluxe edition)  
-- Found a price outlierr in wwe 2k24 since (99.99)  
-- Revenue data based on estimates

## About this data set:
This dataset contains comprehensive information on the top 1500 games released on Steam between January 1, 2024, and September 9, 2024. Aggregated from 30 separate files, and combined into a single dataset. Minor adjustments were made, such as aligning game release dates for consistency.

## Key Features:
Game Details: Includes titles, release dates, and developer/publisher information.
Sales and Revenue: Tracks the number of copies sold, revenue generated, and pricing details.
Player Engagement: Provides average playtime, peak player counts, and other user engagement metrics.
Reviews and Scores: Features review scores and ratings.
Dynamic Market Data: Offers insights into game performance trends over time, such as sales rank and price fluctuations.


In [3]:
import pandas as pd

data = pd.read_csv('./Steam_2024_bestRevenue_1500.csv')


data.sort_values(by='price', ascending=False, inplace=True)

data.head()

,name,releaseDate,copiesSold,price,revenue,avgPlaytime,reviewScore,publisherClass,publishers,developers,steamId
0,WWE 2K24,07-03-2024,165301,99.99,8055097.0,42.365140,71,AAA,2K,Visual Concepts,2315690
597,The Crew Motorfest,18-04-2024,56044,69.99,1902618.0,18.217624,59,AAA,Ubisoft,Ubisoft Ivory Tower,2698940
1473,Like a Dragon: Infinite Wealth,25-01-2024,369330,69.99,21265220.0,38.898038,91,AA,SEGA,Ryu Ga Gotoku Studio,2072450
48,EA SPORTS™ Madden NFL 25,15-08-2024,50521,69.99,3182368.0,16.106491,50,AAA,Electronic Arts,Tiburon,2582560
1454,Dragon's Dogma 2,21-03-2024,1785028,69.99,111478291.0,31.687050,0,AAA,"CAPCOM Co., Ltd.","CAPCOM Co., Ltd.",2054970
